In [1]:
import hublib
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output
import matplotlib
import matplotlib.pyplot as plt
import scipy
import numpy as np
from decimal import Decimal
import unicodedata
from unicodedata import lookup as GL
import sympy as sy
import time
from joblib import Parallel, delayed

In [2]:
class Spectrum:
    def __init__(self):
        self.x=0
        self.y=0
        self.I=[]
        self.W=[]
        self.If=[]
        self.PG=[]
        self.PGp=[]
        self.PD=[]
        self.IDfit=[]
        self.Q=0
        self.diffs=[]
        self.mdi=0
        self.md=0
        self.mf=[]

In [3]:
global Specs
Specs=[]

In [4]:
from hublib.ui import FileUpload

def mycb(name,val):
    #print("CB %s (length = %s bytes)" % (name, len(val)))
    global fnm
    fnm=name

f = FileUpload("Please upload Raman spectra data file (CSV)", 
               "Raman data files should be uploaded as 2 column CSV files",cb=mycb,maxsize=10000000)
f

<IPython.core.display.Javascript object>

Box(children=(HTML(value='<p data-toggle="popover" title="Raman data files should be uploaded as 2 column CSV files">Please upload Raman spectra data file (CSV)</p>', layout=Layout(flex='2 1 auto')), FileWidget(maxsize=10000000)), layout=Layout(border='solid 1px lightgray', display='flex', flex_flow='row', justify_content='space-between', width='auto'))

In [5]:
def errprint(code):
    errfile=pd.read_csv('errfile.txt',sep='\t',header=None)
    with errout:
        clear_output()
        print(errfile[0][code])
        errout

In [6]:
def case_lookup(index):
    casefile=pd.read_csv('Case_List.txt',sep='\t',header=None)
    c=casefile[0][index]
    return c

In [7]:
fit_but = widgets.Button(description='Do Fitting')
    
def fit_but_cb(change):
    
    with errout:
        clear_output()
    with diffsplot:
        clear_output()
    with datplot:
        clear_output()
    with plist:
        clear_output()
        
    f.save('tmpfile')
    if fnm[-3:]=='txt':
        sp='\t'
    elif fnm[-3:]=='csv':
        sp=','
    else:
        errprint(0)
        return
    
    with plist:
        clear_output()
        print('Reading daata file...')
    data = pd.read_csv('tmpfile',sep=sp,header=None)
    with plist:
        clear_output()
        print('Data file read')
    
    n=int(data.size/len(data)) #n determines the size of the data file

    ##Single Spectra Data File, n=2    
    if n==2:
        with plist:
            clear_output()
            print('Fitting single spectra data.')
        
        s=Spectrum()
        Spectra(s,data)
        Fit(s)
        
        dtplot(s)

        with diffsplot:
            clear_output()
            plt.plot(s.diffs,'kv')
            plt.plot(s.mdi,s.md,'gv')
            plt.annotate((round(Decimal(s.md),2)),xy=(s.mdi,1.2*s.md))
            plt.xticks(range(6),('1','2','3','4','5','Graphite'))
            plt.xlabel('# Layers')
            plt.ylabel('$\Delta$ [%]')
            plt.show()

        with plist:
            clear_output()
            G=GL('GREEK CAPITAL LETTER GAMMA')
            o=GL('GREEK SMALL LETTER OMEGA')
            print('G Fitting Parameters:\n\tA=%1.2f\n\t%s=%1.2f\n\t%s=%1.2f\n'
                  'G\' Fitting Parameters:\n\tA=%1.2f\n\t%s=%1.2f\n\t%s=%1.2f\n'
                  'D Fitting Parameters:\n\tA=%1.2f\n\t%s=%1.2f\n\t%s=%1.2f\n'
                  'Quality=%1.2f (Ratio of D to G)\n'
                  'Best Case Match: %s'
                  %(s.PG[0],G,s.PG[1],o,s.PG[2],s.PGp[0],G,s.PGp[1],o,s.PGp[2],s.PD[0],G,s.PD[1],o,s.PD[2],s.Q,case_lookup(s.mdi)))

    #Map files will be much larger than 2 points and need separate handling
    elif n > 2:
        Specs=[]
        Map(data)
    else:
        errprint(0)
        return
    
fit_but.on_click(fit_but_cb)
fit_but

Button(description='Do Fitting', style=ButtonStyle())

In [8]:
def Map(data):
    
    t1=time.clock()
    T1=t1
    
    W=data[:][0:1]
    W=np.array(W)
    W=W[~np.isnan(W)]
    
    x=data[0]
    x=np.array(x)
    x=x[~np.isnan(x)]
    xu=np.unique(x)
    
    y=data[1]
    y=np.array(y)
    y=y[~np.isnan(y)]
    yu=np.unique(y)
    
    n=yu.size*xu.size
    
    s=Spectrum()
    
    Parallel(n_jobs=1)(delayed(maploop)(s,Specs,data,W,x,y,n,k) for k in range(n))
    
    wG=np.transpose(np.array([o.PG for o in Specs]))[2]      
    Mplot(x,y,wG)
    with diffsplot:
        clear_output()
        plt.hist(wG,bins='auto')
        plt.show()
    T2=time.clock()
    with plist:
        clear_output()
        print('Fitting Finished\nActual time [min]: %1.2f'%((T2-T1)/60))
    param.disabled=False

In [9]:
def maploop(s,Specs,data,W,x,y,n,k):
    #for k in range(n):

        s=Spectrum()
        
        I_raw=np.array(data)[k+1][2:1026]
        I=((I_raw-np.min(I_raw))/np.max(I_raw-np.min(I_raw)))
        s.I=I
        s.W=W
        s.x=x[k]
        s.y=y[k]
        Fit(s)
        Specs.append(s)
        
        t1=time.clock()
        t2=time.clock()
        dt=(t2-t1)
        t1=t2
        pdone=100*(k+1)/n
        t_e=(n-k)*dt
        if t_e<60:
            ts='sec'
            t_est=t_e
        else:
            ts='min'
            t_est=t_e/60
        with plist:
            clear_output()
            print('Fitting map data. This may take some time...\n%1.2f%% Done\nEstimated time left [%s]: %1.2f'%(pdone,ts,t_est))
        
        return Specs

In [10]:
def Mplot(x,y,z):
    global p,point,datax
    xi = np.linspace(min(x), max(x))
    yi = np.linspace(min(y), max(y))
    X, Y = np.meshgrid(xi, yi)
    
    with datplot:
        clear_output()
        Z=matplotlib.mlab.griddata(x, y, z, xi, yi, interp='linear') 
        fig = plt.figure() #Plot to choose peaks
        datax = fig.add_subplot(111)
        p,=datax.plot([0],[0])
        point=pickPeaks(p)
        datax.contourf(X,Y,Z)
        plt.set_cmap('inferno')
        plt.xlabel('x [mm]')
        plt.ylabel('y [mm]')
        plt.title(param.index)
        plt.show()

In [11]:
def Spectra(s,data):
    srow=0;
    if type(data[0][0])==str:
        srow=1
    
    W=data[0][srow:len(data)]
    W=np.array(W);W=W.astype(float)
    I_raw=data[1][srow:len(data)]
    I_raw=np.array(I_raw);I_raw=I_raw.astype(float)
    I=((I_raw-np.min(I_raw))/np.max(I_raw-np.min(I_raw)));
    s.I=I
    s.W=W
    return s

In [12]:
def Fit(s):
    W=s.W
    I=s.I
    pG=[np.max(I), 5, 1581.6] #a w b
    pGp=[np.max(I), 18, 2675]
    
    #PG=%octave -i pG,W,I lsqcurvefit(@Single_Lorentz,pG,W,I,[0,0,1400],[max(I),50,2000]);
    #PGp=%octave -i pGp,W,I lsqcurvefit(@Single_Lorentz,pGp,W,I,[0,0,2000],[max(I),100,3000]);
    
    PG=scipy.optimize.curve_fit(Single_Lorentz,W,I,pG,bounds=([0,0,1400],[np.max(I),50,2000]))
    PGp=scipy.optimize.curve_fit(Single_Lorentz,W,I,pGp,bounds=([0,0,2000],[np.max(I),100,3000]))
    
    PG=PG[0][:]
    PGp=PGp[0][:]
    
    PG[1]=np.absolute(PG[1]);PGp[1]=np.absolute(PGp[1]); #FWHM sometimes returns - bc always squared
    
    IGfit=Single_Lorentz(PG[0],PG[1],PG[2],W);
    IGpfit=Single_Lorentz(PGp[0],PGp[1],PGp[2],W);
    IGfit=IGfit#[0]
    IGpfit=IGpfit#[0];
    s.If=IGfit+IGpfit;
    
    s.PG=PG
    s.PGp=PGp
    
    pD=[np.max(I),5,1300]
    #PD=%octave -i pD,W,I lsqcurvefit(@Single_Lorentz,pD,W,I,[0,0,1200],[max(I),50,1400]);
    PD=scipy.optimize.curve_fit(Single_Lorentz,W,I,pD,bounds=([0,0,1200],[np.max(I),50,1400]))
    PD=PD[0][:]
    PD[1]=np.absolute(PD[1]);
    IDfit=Single_Lorentz(PD[0],PD[1],PD[2],W);
    s.IDfit=IDfit#[0]
    Q=1-(PD[0]/PG[0])
    s.Q=Decimal(Q)
    s.PD=PD
    
    Cdat=np.load('Cfits.npy')

    diffs_lin=[];diffs_Gp=[];
    diffs=[];
    for d in range(6):
        
        LG=Cdat[d][0];
        LGp=Cdat[d][1];
        LGfit=Single_Lorentz(LG[0],LG[1],LG[2],W); 
        LGpfit=Single_Lorentz(LGp[0],LGp[1],LGp[2],W);
        Lf=(LGfit+LGpfit)
 
        dfGp=np.average(np.absolute(100*(PGp-LGp)/LGp))
        dfG=np.average(np.absolute(100*(PG-LG)/LG))
        df=np.average([dfGp,dfG])
        diffs.append(df)
    
    s.diffs=diffs
    md=np.min(diffs)
    mdi=np.argmin(diffs)

    mG=Cdat[mdi][0];mGp=Cdat[mdi][1];
    mGfit=Single_Lorentz(mG[0],mG[1],mG[2],W);
    mGpfit=Single_Lorentz(mGp[0],mGp[1],mGp[2],W);
    mf=mGfit+mGpfit;mf=mf#[0]
    s.mf=mf
    s.md=md
    s.mdi=mdi
    return s

In [13]:
def Single_Lorentz(a,w,b,x):
    return a*(((w/2)**2)/(((x-b)**2)+((w/2)**2)))

In [14]:
def dtplot(s):
        with datplot:
            clear_output()
            plt.plot(s.W,s.I,'b',s.W,s.If+s.IDfit,'r',s.W,s.mf,'g')
            plt.xlabel('$\omega$ $[cm^{-1}]$')
            plt.ylabel('$I_{norm}$ [arb]')
            plt.legend(labels=['Raw','Fit','Test'])
            plt.annotate('Q=%1.2f' %round(s.Q,2) ,xy=(np.min(s.W),0.98))
            plt.annotate('D',xy=(0.85*s.PD[2],1.1*s.PD[0]))
            plt.annotate('G',xy=(0.9*s.PG[2],0.95*s.PG[0]))
            plt.annotate('G\'',xy=(0.94*s.PGp[2],0.95*s.PGp[0]))
            plt.show()

In [15]:
def dfplot(s):
        with diffsplot:
            clear_output()
            plt.plot(s.W,s.I,'b',s.W,s.If+s.IDfit,'r',s.W,s.mf,'g')
            plt.xlabel('$\omega$ $[cm^{-1}]$')
            plt.ylabel('$I_{norm}$ [arb]')
            plt.legend(labels=['Raw','Fit','Test'])
            plt.annotate('Q=%1.2f' %round(s.Q,2) ,xy=(np.min(s.W),0.98))
            plt.annotate('D',xy=(0.85*s.PD[2],1.1*s.PD[0]))
            plt.annotate('G',xy=(0.9*s.PG[2],0.95*s.PG[0]))
            plt.annotate('G\'',xy=(0.94*s.PGp[2],0.95*s.PGp[0]))
            plt.show()

In [16]:
datplot=widgets.Output();datplot

Output()

In [17]:
diffsplot=widgets.Output();diffsplot

Output()

In [18]:
errout=widgets.Output();errout

Output()

In [19]:
plist=widgets.Output();plist

Output()

In [20]:
o=GL('GREEK SMALL LETTER OMEGA')
G=GL('GREEK CAPITAL LETTER GAMMA')

def param_change(change):
    d={1: 'PG',2: 'PG',3: 'PG',4: 'PGp',5: 'PGp',6: 'PGp',7: 'PD',8: 'PD',9: 'PD'}
    if param.value in [1,4,7]:
        ind=0
    elif param.value in [2,5,8]:
        ind=1
    elif param.value in [3,6,9]:
        ind=2
    else:
        ind=0
 
    if param.value in [1,2,3]:
        z=np.transpose(np.array([o.PG for o in Specs]))[ind]
    elif param.value in [4,5,6]:
        z=np.transpose(np.array([o.PGp for o in Specs]))[ind]
    elif param.value in [7,8,9]:
        z=np.transpose(np.array([o.PD for o in Specs]))[ind]
    else:
        z=np.transpose(np.array([o.PG for o in Specs]))[2]

    xvals=np.transpose(np.array([o.x for o in Specs]))
    yvals=np.transpose(np.array([o.y for o in Specs]))
    Mplot(xvals,yvals,z)

param=widgets.Dropdown(description='Parameter')
param.options=options={'Select': '', 'I_G': 1, (G+'_G'): 2, (o+'_G'): 3, 'I_G\'': 4, (G+'_G\''): 5, (o+'_G\''): 6, 'I_D': 7, (G+'_D'): 8, (o+'_D'): 9}
param.observe(param_change,names='value')
param.disabled=True
param

Dropdown(description='Parameter', disabled=True, options={'Select': '', 'I_G': 1, 'Γ_G': 2, 'ω_G': 3, "I_G'": 4, "Γ_G'": 5, "ω_G'": 6, 'I_D': 7, 'Γ_D': 8, 'ω_D': 9}, value='')

In [21]:
class pickPeaks:
    def __init__(self, line):
        self.line = line
        self.xs = line.get_xdata()
        self.ys = line.get_ydata()
        self.cid = line.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        print('click', event)
        if event.inaxes!=self.line.axes: return
        self.xs=event.xdata
        self.ys=event.ydata
        with datplot:
            plt.show()
        Map_Spec_Plot()

In [22]:
def Map_Spec_Plot():
    from scipy.spatial import cKDTree
    xvals=np.transpose(np.array([o.x for o in Specs]))
    yvals=np.transpose(np.array([o.y for o in Specs]))
    XY=np.zeros((len(Specs),2))
    XY[:,0]=xvals
    XY[:,1]=yvals
    tree = cKDTree(XY)
    dis, ind = tree.query([point.xs,point.ys], k=1)
    print(ind)
    #dfplot(Specs[ind])